In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
import altair as alt
from ete3 import NCBITaxa
ncbi = NCBITaxa()
#ncbi.update_taxonomy_database()
import numpy as np

In [ ]:
# This file is a VGP supplement table for Galaxy + VGP paper
df = pd.read_excel("../data/vgp_table.xlsx","ST2 - Assembly full list")

# DANGER HERE
This slicing below is very arbitrary and depends on the content of the spreadsheet. Check every time!

In [ ]:
# At present the table is not harmonized, so some processing is required
df = df.iloc[:73,:]

In [ ]:
# If a species only has one assembly, it will be called hap1
df.loc[ ( df['Assembly version'] != 'hap1' ) & ( df['Assembly version'] != 'hap2' ), 'Assembly version'] = 'hap1'

In [ ]:
for sp, taxid in ncbi.get_name_translator(df['NCBI Species'].unique()).items():
    lineage = ncbi.get_lineage(taxid[0])
    for item in lineage:
        rank = ncbi.get_rank([item])
        for key,value in rank.items():
            if value == 'phylum':
                df.loc[ df['NCBI Species'] == sp ,'phylum'] = ncbi.translate_to_names([key])[0]
            elif value == 'class':
                df.loc[ df['NCBI Species'] == sp ,'class_']  = ncbi.translate_to_names([key])[0]
            elif value == 'order':
                df.loc[ df['NCBI Species'] == sp ,'order']  = ncbi.translate_to_names([key])[0]

# DANGER HERE! 
For some stupid reason there is no `class` rank for Reptilia, so these need to be set manually:

In [ ]:
df.loc[df['class_'].isna(), 'class_'] = "Reptilia"

In [ ]:
for _ in df.columns: print(_)

In [ ]:
for _ in df['class_'].unique(): print(_)

In [ ]:
# Phylo rank for prper ordering of classes:
class_rank = {
    'Actinopteri':2,
    'Amphibia':3,
    'Aves':6,
    'Chondrichthyes':1,
    'Lepidosauria':5,
    'Mammalia':7,
    'Reptilia':4
}

In [ ]:
for key in class_rank:
    df.loc[df['class_']==key, 'c_rank'] = class_rank[key]

In [ ]:
df = df.sort_values(by=['c_rank','order','NCBI Species','Assembly version'],ignore_index = True).reset_index()

# DANGER HERE

The following three `groupby`'s need to be converted to a function. They are doing slightkly different things = hence the repeatitive code

In [ ]:
df_for_order_chart = df.groupby(
        'order'
    ).agg(
        mn = pd.NamedAgg(column='index',aggfunc ='min'),
        mx = pd.NamedAgg(column='index',aggfunc = lambda x: x.max()+1),
        mid = pd.NamedAgg(column='index',aggfunc = lambda x: (x.min()+((x.max()-x.min())/2))),
        class_ = pd.NamedAgg(column='class_',aggfunc = max)
    ).reset_index()

In [ ]:
df_for_order_chart.head()

In [ ]:
df_for_class_chart = df.groupby(
        'class_'
    ).agg(
        mn = pd.NamedAgg(column='index',aggfunc ='min'),
        mx = pd.NamedAgg(column='index',aggfunc = lambda x: x.max()+1),
        mid = pd.NamedAgg(column='index',aggfunc = lambda x: (x.min()+((x.max()-x.min())/2))),
    ).reset_index()

In [ ]:
df_for_class_chart.head()

In [ ]:
df_for_species_chart = df.groupby(
        ['NCBI Species','Assembly version']
    ).agg(
        sp = pd.NamedAgg(column='Species',aggfunc =lambda x: (x.max().replace(' ','_'))),
        mn = pd.NamedAgg(column='index',aggfunc ='min'),
        mx = pd.NamedAgg(column='index',aggfunc = lambda x: x.max()+1),
        mid = pd.NamedAgg(column='index',aggfunc = lambda x: (x.min()+((x.max()-x.min())/2))),
        species = pd.NamedAgg(column='NCBI Species',aggfunc = max),
        class_ = pd.NamedAgg(column='class_',aggfunc = max),
        order = pd.NamedAgg(column='order',aggfunc = max),
        size = pd.NamedAgg(column='Genome size¹',aggfunc = lambda x: (x.max()/1000000)),
        het = pd.NamedAgg(column='Heterozygosity¹',aggfunc = max),
        rep = pd.NamedAgg(column='Repeat content¹',aggfunc = max),
        s_ng50 = pd.NamedAgg(column='Scaffold NG50',aggfunc = lambda x: (x.max()/1000000)),
        c_ng50 = pd.NamedAgg(column='Contig NG50',aggfunc = lambda x: (x.max()/1000000)),
        sGap = pd.NamedAgg(column='Total gap length in scaffolds',aggfunc = lambda x: (x.max()/1000000)),
        merqComp = pd.NamedAgg(column='Merqury completeness (hifi kmers) of contigs in BOTH HAPS (if pri/alt, then post-purging)',aggfunc = max),
        buscoComp = pd.NamedAgg(column='%age complete BUSCO genes in contigs (if pri/alt, then post-purging)',aggfunc = max),
    ).reset_index()

In [ ]:
df_for_species_chart

In [ ]:
# This is super-ugly
# It needs to be partitioned into separate functions

domain_c = df['class_'].unique()
range_c = ['#d73027','#fc8d59','#fee090','#ffffbf','#e0f3f8','#91bfdb','#4575b4']

class_chart = alt.Chart(df_for_class_chart).mark_rect(opacity=1,stroke='black', strokeWidth=.5).encode(
    y = alt.Y('mn:Q',scale=alt.Scale(domain=[0, 72],nice=False),axis=None),
    y2 = 'mx:Q',
    color=alt.Color('class_:N',scale=alt.Scale(domain=domain_c,range=range_c),legend=None),
).properties(
        height=900,
        width=100,
        title = 'Class'
)

class_text = class_chart.transform_calculate(
    link='https://www.google.com/search?q=' + alt.datum.class_
).mark_text(color='black',align='center',baseline="middle",fontSize=12,fontWeight="bold",dy=-5).encode(
    y = 'mid:Q',
    text='class_:N',
    color=alt.value('black'),
    href = 'link:N',
    #color=alt.condition(
    #    alt.datum['name']== "Reptilia",
    #    alt.value("white"),
    #    alt.value("black")
    #)
)

order_chart = alt.Chart(df_for_order_chart).mark_rect(opacity=1,stroke='black', strokeWidth=.5).encode(
    y = alt.Y('mn:Q',scale=alt.Scale(domain=[0, 72],nice=False),axis=None),
    y2 = 'mx:Q',
    color=alt.Color('class_:N',scale=alt.Scale(domain=domain_c,range=range_c),legend=None),
    opacity=alt.Opacity('order:N',legend=None)
).properties(
        height=900,
        width=100,
    title = 'Order'
)

order_text = order_chart.transform_calculate(
    link='https://www.google.com/search?q=' + alt.datum.order
).mark_text(color='black',align='center',baseline="middle",fontSize=10,dy=-5).encode(
    y = 'mid:Q',
    text='order:N',
    color=alt.value('black'),
    opacity=alt.value(1),
    href = 'link:N',
    #color=alt.condition(
    #    alt.datum['name']== "Reptilia",
    #    alt.value("white"),
    #    alt.value("black")
    #)
)

species_chart = alt.Chart(df_for_species_chart).mark_rect(opacity=1,stroke='black', strokeWidth=.5).encode(
    y = alt.Y('mn:Q',scale=alt.Scale(domain=[0, 72],nice=False),axis=None),
    y2 = 'mx:Q',
    #x = alt.value(10)
    color=alt.Color('class_:N',scale=alt.Scale(domain=domain_c,range=range_c),legend=None),
    opacity=alt.Opacity('order:N',legend=None)
).properties(
        height=900,
        width=350,
    title='Species'
)

species_text = species_chart.transform_calculate(
    link='https://genomeark.github.io/genomeark-all/' + alt.datum.sp
).mark_text(color='black',align='center',baseline="middle",fontSize=8,dy=-5,fontStyle="italic").encode(
    y = 'mid:Q',
    text='species:N',
    href = 'link:N',
    #color=alt.value('black'),
    opacity=alt.value(1),
    color=alt.condition(
        alt.datum['class_']== "Mammalia",
        alt.value("black"),
        alt.value("black")
    )
)

In [ ]:
def hmt(df,col,y,y2,op,scheme,m_value,format_,wid,title,stitle):
    chart = alt.Chart(df,width=wid).mark_rect().encode(
        y = alt.Y(y,scale=alt.Scale(domain=[0, 72],nice=False),axis=None),
        y2 = y2,
    color=alt.Color(col,scale=alt.Scale(scheme=scheme),legend=None),
    ).properties(
        height=900,
        #width=50,
        title={ "text": title,"subtitle":stitle }
               
    )
    
    text = chart.mark_text(color='black',align='center',baseline="middle",fontSize=8,dy=-5).encode(
    y = y,
    text=alt.Text(col,format=format_),
    #color=alt.value('black'),
    opacity=alt.value(1),
    color=alt.condition(
        alt.datum[col] > m_value,
        alt.value("white"),
        alt.value("black")
    )
    )
    return(chart + text)

In [ ]:
size = hmt(df_for_species_chart,'size','mn:Q','mx:Q','order','goldred',4000,",.0f",60,'Size','(Mb)')

In [ ]:
het = hmt(df_for_species_chart,'het','mn:Q','mx:Q','order','yellowgreen',.5,",.2f",60,'Het','(%)')

In [ ]:
rep = hmt(df_for_species_chart,'rep','mn:Q','mx:Q','order','yellowgreen',50,",.1f",60,'Repeat','(%)')

In [ ]:
s_ng50 = hmt(df_for_species_chart,'s_ng50','mn:Q','mx:Q','order','goldred',500,",.0f",60,'Scaffold NG50','(Mb)')

In [ ]:
c_ng50 = hmt(df_for_species_chart,'c_ng50','mn:Q','mx:Q','order','goldred',60,",.0f",60,'Contig NG50','(Mb)')

In [ ]:
sGap = hmt(df_for_species_chart,'sGap','mn:Q','mx:Q','order','goldred',50,",.0f",60, 'Gaps', '(Mb)')

In [ ]:
merqComp = hmt(df_for_species_chart,'merqComp','mn:Q','mx:Q','order','yellowgreen',50,",.2f",60, 'Merqury', '(%)')

In [ ]:
buscoComp = hmt(df_for_species_chart,'buscoComp','mn:Q','mx:Q','order','yellowgreen',50,",.2f",60, 'Busco', '(%)')

In [ ]:
names = alt.hconcat( (class_chart + class_text ),( order_chart + order_text ),( species_chart + species_text))
stats = alt.hconcat( size, c_ng50, s_ng50, sGap,het, rep, merqComp, buscoComp )

In [ ]:
species_stats = (names | stats).configure_concat(
    spacing=0
).configure_title(fontSize=9,subtitleFontSize=8)

In [ ]:
species_stats

In [ ]:
# Save chart
from altair_saver import save
species_stats.save("../json/species_stats.vl.json")
#save(species_stats,"../svg/species_stats.svg")